In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from types import SimpleNamespace
from pathlib import Path
from datetime import datetime
import math
import os
import re

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import StratifiedKFold, GroupKFold
from sklearn.metrics import log_loss,f1_score
from sklearn.model_selection import train_test_split


from fastai.imports import *
from torch.utils.data import DataLoader
import warnings,transformers,logging,torch
from transformers import TrainingArguments,Trainer
from transformers import AutoModelForSequenceClassification,AutoTokenizer
import datasets
from datasets import load_dataset, Dataset, DatasetDict

In [ ]:
seed = 42
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)

In [ ]:
warnings.simplefilter('ignore')
logging.disable(logging.WARNING)

In [ ]:
#os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
INPUT_DIR = '/content/drive/MyDrive/Competitions/Signate/Student Cup 2022/input/'
OUTPUT_DIR = '/content/drive/MyDrive/Competitions/Signate/Student Cup 2022/output/'
OUTPUT_SUB_DIR = os.path.join(OUTPUT_DIR,'Submission')
OUTPUT_MODEL_DIR = os.path.join(OUTPUT_DIR,'Model')

In [ ]:
model_nm = 'microsoft/deberta-v3-small'

In [ ]:
train = pd.read_csv(os.path.join(INPUT_DIR, 'train.csv'))
test = pd.read_csv(os.path.join(INPUT_DIR, 'test.csv'))
submission_df = pd.read_csv(os.path.join(INPUT_DIR, 'submit_sample.csv'),header=None)
display(train.head())
print(train.shape)

,id,description,jobflag
0,0,<li>Develop cutting-edge web applications that perform superbly across all platforms.</li><li>Work in a highly collaborative environment with cross-functional teams on projects ranging from a few weeks to a few months in length.</li><li>Maintain high standard of quality as you creatively and strategically problem solve throughout the product delivery process.</li><li>Be able to effectively communicate your work with both technical and non-technical peers</li><li>Be excited about new web technologies/techniques</li><li>Build solid front-end architectures that integrate easily with other sys...,3
1,1,"<li> Designs and develops high quality, scalable and efficient solutions and products on schedule </li><li> Engages with system users and business analysts to identify system enhancements and/or new applications to meet business needs. </li><li> Actively contributes to the development of solutions and ideas that add value. </li><li> Proactively performs extensive system testing to ensure that the systems work efficiently and are developed following the applicable development methodology. </li><li> Provides support during meetings as required; prepares and takes dictation, minutes, agendas,...",3
2,2,"<li>Functions as a point person for Network Strategy work. Represents Network Strategy at internal and external stakeholder meetings. Serves as a resource for functional areas in responding to Network Strategy initiatives (identifying standard or historical responses) and coordinates response with other departments as necessary.</li><li>Communicates effectively and serves as a point of contact for status updates and overall project status. Coordinates activity with functional areas, lines of business, and markets to ensure schedule is met.</li><li>Monitors network activities to ensure the ...",4
3,3,"<li> Work on the technical design, development, release and deployment of cloud-based infrastructure and applications </li><li> Working with, and supporting, the development team with application configuration for deployment, monitoring and other automation </li><li> Provide operational management and support of Linux and Windows servers and containers, including server hardening, patching, network security and log management to deliver web application and service stacks </li><li> Mentor and collaborate with other team members</li><li> Provide on-call/out of hours support, as part of a rot...",3
4,4,"<li>Quantify the resources required for a task/project related to an Enterprise Resource Platform (ERP) implementation</li><li>Familiarize yourself with integration involving Sales, CRM, eCommerce, supply chain, manufacturing, inventory, POS, accounting, and so much more</li>",4


(1516, 3)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_nm)
#tokz.model_max_length = 512
sep = tokenizer.sep_token
sep

'[SEP]'

In [ ]:
tokenizer.all_special_tokens

['[CLS]', '[SEP]', '[UNK]', '[PAD]', '[MASK]']

In [ ]:
def remove_tag(x):
    p = re.compile(r"<[^>]*?>")
    return p.sub('',x)

def cleaning(texts):
    clean_texts = []
    for text in texts:
        # htmlタグを削除
        text = remove_tag(text)
        #アルファベット以外をスペースに置き換え
        #clean_punc = re.sub(r'[^a-zA-Z]', ' ', text)
        clean_texts.append(text)
    return clean_texts

train['inputs'] = cleaning(train['description'])
test['inputs'] = cleaning(test['description'])
train = train.rename(columns = {"jobflag": "label"})
train["label"] = train["label"].apply(lambda x : 0 if x == 4 else x)
train

,id,description,label,inputs
0,0,<li>Develop cutting-edge web applications that perform superbly across all platforms.</li><li>Work in a highly collaborative environment with cross-functional teams on projects ranging from a few weeks to a few months in length.</li><li>Maintain high standard of quality as you creatively and strategically problem solve throughout the product delivery process.</li><li>Be able to effectively communicate your work with both technical and non-technical peers</li><li>Be excited about new web technologies/techniques</li><li>Build solid front-end architectures that integrate easily with other sys...,3,Develop cutting-edge web applications that perform superbly across all platforms.Work in a highly collaborative environment with cross-functional teams on projects ranging from a few weeks to a few months in length.Maintain high standard of quality as you creatively and strategically problem solve throughout the product delivery process.Be able to effectively communicate your work with both technical and non-technical peersBe excited about new web technologies/techniquesBuild solid front-end architectures that integrate easily with other systems and technologiesWorking closely with other d...
1,1,"<li> Designs and develops high quality, scalable and efficient solutions and products on schedule </li><li> Engages with system users and business analysts to identify system enhancements and/or new applications to meet business needs. </li><li> Actively contributes to the development of solutions and ideas that add value. </li><li> Proactively performs extensive system testing to ensure that the systems work efficiently and are developed following the applicable development methodology. </li><li> Provides support during meetings as required; prepares and takes dictation, minutes, agendas,...",3,"Designs and develops high quality, scalable and efficient solutions and products on schedule Engages with system users and business analysts to identify system enhancements and/or new applications to meet business needs. Actively contributes to the development of solutions and ideas that add value. Proactively performs extensive system testing to ensure that the systems work efficiently and are developed following the applicable development methodology. Provides support during meetings as required; prepares and takes dictation, minutes, agendas, notices and manages mailings of meeting..."
2,2,"<li>Functions as a point person for Network Strategy work. Represents Network Strategy at internal and external stakeholder meetings. Serves as a resource for functional areas in responding to Network Strategy initiatives (identifying standard or historical responses) and coordinates response with other departments as necessary.</li><li>Communicates effectively and serves as a point of contact for status updates and overall project status. Coordinates activity with functional areas, lines of business, and markets to ensure schedule is met.</li><li>Monitors network activities to ensure the ...",0,"Functions as a point person for Network Strategy work. Represents Network Strategy at internal and external stakeholder meetings. Serves as a resource for functional areas in responding to Network Strategy initiatives (identifying standard or historical responses) and coordinates response with other departments as necessary.Communicates effectively and serves as a point of contact for status updates and overall project status. Coordinates activity with functional areas, lines of business, and markets to ensure schedule is met.Monitors network activities to ensure the quality and timeliness..."
3,3,"<li> Work on the technical design, development, release and deployment of cloud-based infrastructure and applications </li><li> Working with, and supporting, the development team with application configuration for deployment, monitoring and other automation </li><li> Provide operational management and support of Linux and Windows servers and c

In [ ]:
ds = Dataset.from_pandas(train)

In [ ]:
def tok_func(x): return tokenizer(x["inputs"], truncation=True)

In [ ]:
tok_func(ds[0])

{'input_ids': [1, 16323, 2947, 271, 9676, 967, 1567, 272, 2091, 33618, 679, 305, 3938, 260, 22625, 267, 266, 1344, 7641, 1192, 275, 1943, 271, 21414, 1737, 277, 1205, 5170, 292, 266, 477, 1033, 264, 266, 477, 740, 267, 2231, 260, 112619, 459, 1264, 265, 607, 283, 274, 21665, 263, 16154, 735, 3634, 1075, 262, 714, 1779, 568, 260, 10374, 526, 264, 2864, 4105, 290, 374, 275, 462, 2064, 263, 745, 271, 27119, 7295, 10374, 2199, 314, 353, 967, 2737, 320, 80222, 268, 36717, 2176, 831, 271, 3308, 30805, 272, 7424, 1166, 275, 340, 1050, 263, 2737, 42212, 3390, 275, 340, 9990, 287, 17345, 271, 3308, 261, 21373, 261, 2169, 261, 24589, 285, 20330, 266, 4197, 1251, 264, 2353, 263, 266, 986, 1211, 264, 1729, 320, 558, 311, 2620, 280, 268, 451, 374, 886, 261, 500, 1943, 271, 51515, 263, 1943, 271, 22028, 5309, 263, 876, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
tok_ds = ds.map(tok_func, batched=True, remove_columns=("id","inputs","description"))

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
tok_ds[0].keys()

dict_keys(['label', 'input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=2022)

In [ ]:
for trn_idxs, val_idxs in skf.split(train,train.label):
  print(len(trn_idxs),len(val_idxs))
  break

1212 304


In [ ]:
dds = DatasetDict({"train":tok_ds.select(trn_idxs),
             "test": tok_ds.select(val_idxs)})

In [ ]:
def get_dds(df, train=True):
    ds = Dataset.from_pandas(df)
    to_remove = ["id","inputs","description"]
    tok_ds = ds.map(tok_func, batched=True, remove_columns=to_remove)
    if train:
        return DatasetDict({"train":tok_ds.select(trn_idxs), "test": tok_ds.select(val_idxs)})
    else: 
        return tok_ds

In [ ]:
lr,bs = 8e-5,16
wd,epochs = 0.01,5

In [ ]:
from sklearn.metrics import log_loss
import torch.nn.functional as F
def score(preds): return {'f1_score': f1_score(preds.label_ids, np.argmax(F.softmax(torch.Tensor(preds.predictions)),axis=1),average='macro')}  

In [ ]:
def get_trainer(dds):
    args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
        evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
        num_train_epochs=epochs, weight_decay=wd, report_to='none')
    model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=4)
    return Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                   tokenizer=tokenizer, compute_metrics=score)

In [ ]:
trainer = get_trainer(dds)
trainer.train()

Epoch,Training Loss,Validation Loss,F1 Score
1,No log,1.042726,0.443993
2,No log,0.826977,0.601790
3,No log,0.922346,0.640924
4,No log,0.936882,0.638742
5,No log,0.949418,0.651695


TrainOutput(global_step=380, training_loss=0.6573892693770559, metrics={'train_runtime': 138.1468, 'train_samples_per_second': 43.866, 'train_steps_per_second': 2.751, 'total_flos': 411252621457440.0, 'train_loss': 0.6573892693770559, 'epoch': 5.0})

In [ ]:
test_ds = get_dds(test,train=False)

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
preds = F.softmax(torch.Tensor(trainer.predict(test_ds).predictions)).numpy().astype(float)
preds

array([[0.01671763, 0.97867179, 0.00277586, 0.00183473],
       [0.93264538, 0.0571702 , 0.00108564, 0.0090988 ],
       [0.00429193, 0.00941572, 0.01121425, 0.97507817],
       ...,
       [0.00381616, 0.00819402, 0.01034811, 0.97764176],
       [0.66164505, 0.33010697, 0.00152274, 0.0067252 ],
       [0.00788598, 0.90616971, 0.07474703, 0.01119729]])

In [ ]:
pred = np.argmax(preds,axis=1)
pred

array([1, 0, 3, ..., 3, 0, 1])

In [ ]:
sub = submission_df.copy()
sub.columns = ["id","label"]
sub["label"] = pred
sub["label"] = sub["label"].apply(lambda x : 4 if x==0 else x)
sub.to_csv(os.path.join(OUTPUT_SUB_DIR,"submission01.csv"),index=False,header=False)
sub

,id,label
0,1516,1
1,1517,4
2,1518,3
3,1519,4
4,1520,3
...,...,...
1512,3028,3
1513,3029,1
1514,3030,3
1515,3031,4
